# Session 10: ILSVRC

In this notebook we introduce another dataset that has ten categories. We 
will talk about ILSVRC and its role in image analysis.

![ilsvrc](https://cdn-images-1.medium.com/max/1600/1*ci-wnR1A-F_RlCC_FJ6rfw.jpeg)

## Setup

We need to load the modules within each notebook. Here, we load the
same set as in the previous question.

In [ ]:
%pylab inline

import numpy as np
import scipy as sp
import pandas as pd
import sklearn
from sklearn import linear_model
import urllib

import os
from os.path import join

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.rcParams["figure.figsize"] = (8,8)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.optimizers import SGD, RMSprop

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

## Imagenette

We are now going to look at a larger dataset with 10 different
classes. Our goal is to use neural networks to classify which
of the categories an image comes from. As usual, we have a
spreadsheet showing the metadata for each image.

In [ ]:
df = pd.read_csv(join("..", "data", "imagenette.csv"))
df

Here are the ten categores of images in the dataset.

In [ ]:
import collections

class_names = np.array(list(collections.OrderedDict.fromkeys(df.class_name.values)))
class_names

Let's look at some of the data in the corpus. Can you tell which of the
categories each image is from.

In [ ]:
plt.figure(figsize=(14, 14))

idx = np.random.permutation(range(len(df)))[:15]

for ind, i in enumerate(idx):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(5, 3, ind + 1)

    img = imread(join('..', 'images', 'imagenette', df.filename[i]))
    plt.imshow(img)
    plt.axis("off")

## Loading the dataset

As before, we also have to read in the dataset using a tandardized image size.

In [ ]:
img_list = []

for i in range(len(df)):
    img_path = join("..", "images", "imagenette", df.filename[i])
    img = image.load_img(img_path, target_size=(32, 32))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    img_list.append(x)
    
X = np.vstack(img_list) / 255
y = np.int32(df.class_num)

In [ ]:
X.shape

And here is a model similiar to the one from the last session. 
Notice that the number of parameters is much higher.

In [ ]:
model = Sequential()
model.add(Conv2D(32, input_shape=X.shape[1:], kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=10, activation="softmax"))

In [ ]:
model.summary()

The next few steps are also similar to the previous session. We compile
the model, construct training/testing sets, and fit the dataset.

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=SGD(lr=0.03, momentum=0.8, decay=0.0, nesterov=True),
              metrics=['accuracy'])

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

In [ ]:
help(sklearn.model_selection.train_test_split)

In [ ]:
X_train.dtype

In [ ]:
model.fit(X_train, y_train, epochs=25, batch_size=32,
          validation_data=(X_test, y_test))

How well does the model produce predictions?

What categories are the hardest to tell apart?

In [ ]:
from sklearn.metrics import confusion_matrix

yhat = model.predict_classes(X_test)
pd.crosstab(class_names[yhat], class_names[y_test])

What images are not working as expected?

In [ ]:
plt.figure(figsize=(14, 14))

yhat = model.predict_classes(X)

idx = np.where((yhat != y) & (y == 3))[0][:15]

for ind, i in enumerate(idx):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(5, 3, ind + 1)

    img = imread(join('..', 'images', 'imagenette', df.filename[i]))
    plt.imshow(img)
    plt.text(0.5, 0.5, class_names[yhat[i]], fontsize=24, color="orange")
    plt.axis("off")